In [ ]:
print("a")
import pandas as pd

from mainnet_launch.constants import *
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.helpers.update_blocks import (
    get_block_by_timestamp_defi_llama,
    get_block_by_timestamp_alchemy,
)


from mainnet_launch.data_fetching.get_state_by_block import safe_normalize_6_with_bool_success, get_state_by_one_block
from multicall import Call
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


def _fetch_on_chain_spot_prices(row: pd.Series) -> dict:
    try:
        block = get_block_by_timestamp_alchemy(int(row["unix_timestamp"]), ETH_CHAIN, closest="before")
        found_timestamp = ETH_CHAIN.client.eth.get_block(block)

        buy_token_price_call = Call(
            ROOT_PRICE_ORACLE(ETH_CHAIN),
            ["getPriceInQuote(address,address)(uint256)", row["buyToken"], USDC(ETH_CHAIN)],
            [("buy_token_price", safe_normalize_6_with_bool_success)],
        )

        sell_token_price_call = Call(
            ROOT_PRICE_ORACLE(ETH_CHAIN),
            ["getPriceInQuote(address,address)(uint256)", row["sellToken"], USDC(ETH_CHAIN)],
            [("sell_token_price", safe_normalize_6_with_bool_success)],
        )

        prices = get_state_by_one_block([buy_token_price_call, sell_token_price_call], block, ETH_CHAIN)

        row_as_dict = row.to_dict()
        row_as_dict.update(prices)
        row_as_dict["block"] = block
        row_as_dict["found_timestamp"] = found_timestamp.timestamp
        row_as_dict["prices_success"] = True
    except Exception as e:
        row_as_dict = row.to_dict()
        row_as_dict["prices_success"] = False

    return row_as_dict

df = pd.read_csv("combined_swap_quotes.csv")
df["unix_timestamp"] = df["datetime_received"].apply(lambda x: int(pd.to_datetime(x, utc=True).timestamp()))

with ThreadPoolExecutor(max_workers=50) as executor:
    results = list(tqdm(executor.map(_fetch_on_chain_spot_prices, [row for _, row in df.iterrows()]), total=len(df)))

results_df = pd.DataFrame(results)
results_df.to_csv("swap_qutoes_with_prices.csv", index=False)

a


/Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/web3/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [ ]:
price_df = pd.read_csv("swap_qutoes_with_prices.csv")[
    [
        "datetime_received",
        "label",
        "sell_amount_norm",
        "buy_amount_norm",
        "min_buy_amount_norm",
        "buy_amount_price",
        "min_buy_amount_price",
        "buy_symbol",
        "sell_symbol",
        "label",
        "batch_id",
        "unix_timestamp",
        "buy_token_price",
        "sell_token_price",
        "block",
        "found_timestamp",
    ]
]

SyntaxError: invalid syntax (2826286916.py, line 20)

In [ ]:
price_df["safe_value_bought"] = price_df.apply(lambda row: row["buy_token_price"] * row["buy_amount_norm"], axis=1)
price_df["safe_value_min_bought"] = price_df.apply(
    lambda row: row["buy_token_price"] * row["min_buy_amount_norm"], axis=1
)
price_df["safe_value_sold"] = price_df.apply(lambda row: row["sell_token_price"] * row["sell_amount_norm"], axis=1)
price_df["safe_value_slippage_bps"] = price_df.apply(
    lambda row: 1_000 * (row["safe_value_sold"] - row["safe_value_bought"]) / row["safe_value_sold"], axis=1
)
price_df

,Unnamed: 0.1,Unnamed: 0,buyAmount,minBuyAmount,aggregatorName,datetime_received,3rd_party_response_success,chainId,systemName,slippageBps,...,unix_timestamp,buy_token_price,sell_token_price,block,found_timestamp,safe_value_bought,safe_value_min_bought,safe_value_sold,safe_value_slippage,safe_value_slippage_bps
0,200,50,249345508777,249258237849,Lifi,2025-09-23 19:05:24.563617+00:00,True,1,gen3,3.5,...,1758654324,1.000000,0.999628,23427708,1758654335,249345.508777,249258.237849,249907.00,0.00035,2.246801
1,201,51,149939371209771150475264,149886892429847730572598,Lifi,2025-09-23 19:05:25.836559+00:00,True,1,gen3,3.5,...,1758654325,0.997628,1.000000,23427708,1758654335,149583.715021,149531.360721,150000.00,0.00035,2.775233
2,202,52,249595278540,249507920193,Lifi,2025-09-23 19:05:27.070354+00:00,True,1,gen3,3.5,...,1758654327,1.000000,1.001408,23427708,1758654335,249595.278540,249507.920193,250352.00,0.00035,3.022630
3,203,53,149527257522,149474922982,Lifi,2025-09-23 19:05:28.322472+00:00,True,1,gen3,3.5,...,1758654328,1.000000,0.999873,23427708,1758654335,149527.257522,149474.922982,149980.95,0.00035,3.025001
4,204,54,99699627725,99664732855,Lifi,2025-09-23 19:05:30.564047+00:00,True,1,gen3,3.5,...,1758654330,1.000798,0.999628,23427708,1758654335,99779.188028,99744.265312,99962.80,0.00035,1.836803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,295,145,249216036922,249128811309,Lifi,2025-09-23 19:07:23.346866+00:00,True,1,gen3,3.5,...,1758654443,1.000000,0.999873,23427717,1758654443,249216.036922,249128.811309,249968.25,0.00035,3.009234
96,296,146,49901362356897844297728,49883896880072930052224,Lifi,2025-09-23 19:07:24.623060+00:00,True,1,gen3,3.5,...,1758654444,0.999873,1.000000,23427718,1758654455,49895.024884,49877.561625,50000.00,0.00035,2.099502
97,297,147,99791797337231423000000,99756870208163392001950,Lifi,2025-09-23 19:07:25.866307+00:00,True,1,gen3,3.5,...,1758654445,0.999628,1.000798,23427718,1758654455,99754.674789,99719.760652,100079.80,0.00035,3.248660
98,298,148,49760338089210286178304,49742921970879062578142,Lifi,2025-09-23 19:07:27.060980+00:00,True,1,gen3,3.5,...,1758654447,0.999628,0.997628,23427718,1758654455,49741.827243,49724.417604,49881.40,0.00035,2.798092
